In [18]:
import warnings
warnings.simplefilter(action='ignore')
import os
import sys
parent_dir = os.path.join(os.getcwd(), '..')
if parent_dir not in sys.path: sys.path.append(parent_dir)
import pandas as pd
from sklearn.metrics import root_mean_squared_log_error
from modules.utility import get_date_details, one_hot, upload
from config import args

In [19]:
OUTPUT_FILE_NAME = "sample_submission_average_guessing1.csv"

In [20]:
df = pd.read_csv(args.train_csv)
test_df = pd.read_csv(args.test_csv)

In [21]:
df['date'] = pd.to_datetime(df['date'])

new_date = get_date_details(df)
new_df, dummies = one_hot(new_date)
combined = pd.concat([new_df, dummies], axis=1)

In [22]:
test_df['date'] = pd.to_datetime(test_df['date'])
test_new_date = get_date_details(test_df)
test_new_df, test_dummies = one_hot(test_new_date)
test_combined = pd.concat([test_new_df, test_dummies], axis=1)

In [23]:
combined['d'] = combined['date'].dt.day

In [24]:
tmp = combined.groupby(["store_nbr", *dummies.columns.tolist(), "d"]).agg({'sales': "mean"})


In [25]:
def average_guessor(train_df, test_x):
    train_df = train_df.copy()
    test_x = test_x.copy()
    test_x['d'] = test_x['date'].dt.day
    
    target_mask = train_df['date'] - pd.Timedelta(days=30)
    train_df = train_df[train_df['date'] > target_mask]
    cols = ["store_nbr", *dummies.columns.tolist(), "d"]
    tmp = train_df.groupby(cols).agg({'sales': "mean"})
    tmp = tmp.reset_index()

    pred = test_x.merge(tmp, on=cols, how="left")
    return pred

In [26]:

from sklearn.model_selection import train_test_split
x, y = combined.drop('sales', axis=1), combined['sales']
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.2)



In [27]:
pred = average_guessor(pd.concat([train_x, train_y], axis=1), test_x)

In [28]:

root_mean_squared_log_error(round(pred['sales'].clip(0)), test_y)

1.53494580192939

In [29]:
pred = average_guessor(pd.concat([train_x, train_y], axis=1), test_combined)

In [30]:
upload(
    pred=round(pred['sales'].clip(0)), 
    test_df=test_df, 
    filename=os.path.join(args.output, OUTPUT_FILE_NAME)
)